In [ ]:
# Faiss-GPU가 Colab에서 가장 빠릅니다.
!pip install faiss-gpu
# FaceNet과 MTCNN을 포함하는 라이브러리 설치
!pip install facenet-pytorch
# Pillow(PIL) 최신 버전 확인
!pip install Pillow --upgrade

ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 113.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 117.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB

In [ ]:
# ----------------------------------------------------
# 💡 코사인 거리 -> 유사도 퍼센트 변환 적용된 최종 코드
# ----------------------------------------------------

# 라이브러리 설치 (새 런타임에서 다시 설치 필요)
!pip install numpy==1.23.5 --quiet
!pip install tensorflow opencv-python gradio==3.50.2 pillow scikit-learn --quiet --upgrade

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16
import numpy as np
from pathlib import Path
import pickle
from scipy.spatial.distance import cosine
import gradio as gr
from PIL import Image
import os
import random
import shutil

# 🚨 한글 이름 매핑 사전
KOREAN_NAMES = {'karina': '카리나', 'winter': '윈터', 'giselle': '지젤', 'ningning': '닝닝'}

# ----------------------------------------------------
# 0. 유틸리티 함수
# ----------------------------------------------------
# 코사인 거리를 유사도 퍼센트로 변환하는 함수 (0% ~ 100%)
def distance_to_similarity_percent(distance):
    """코사인 거리를 코사인 유사도 퍼센트로 변환합니다."""
    # 코사인 거리 (0 ~ 2) -> 코사인 유사도 (1 ~ -1) -> 유사도 (0 ~ 2)
    # VGG16 임베딩은 L2 정규화되어 코사인 거리가 0 ~ 2 범위에 있으므로,
    # 유사도(1 - 거리)를 사용하기보다는, 거리가 0에 가까울수록 높은 점수를 주는 방식 사용

    # 💡 L2 정규화된 VGG16 임베딩 벡터 간의 코사인 거리는 일반적으로 0에서 2 사이의 값을 가집니다.
    # 코사인 유사도 = 1 - 코사인 거리 (L2 정규화된 벡터의 경우 0 ~ 2 거리에 해당)

    # 여기서는 직관적으로 1 - (거리/2) * 100% 공식을 사용하겠습니다.
    # (거리가 0일 때 100%, 거리가 2일 때 0%에 가까워지도록)

    # 1 - (거리 / 2) -> 0.0 ~ 1.0 스케일
    similarity = 1 - (distance / 2.0)

    # 0% ~ 100% 변환
    return max(0.0, min(100.0, similarity * 100.0))


# ----------------------------------------------------
# 1. Drive 연결 및 Colab 임시 저장소 복사 (파일 시스템 안정화)
# ----------------------------------------------------
try:
    from google.colab import drive
    drive.mount('/content/gdrive', force_remount=True)
    DRIVE_PHOTO_DIR = Path("/content/gdrive/MyDrive/aespa/aespa_photo")
    COLAB_TEMP_DIR = Path("/content/temp_aespa_photos")
    COLAB_TEMP_DIR.mkdir(exist_ok=True)

    # OSErrors 우회를 위해 Colab 로컬로 복사
    for member_dir in DRIVE_PHOTO_DIR.iterdir():
        if member_dir.is_dir():
            target_dir = COLAB_TEMP_DIR / member_dir.name
            target_dir.mkdir(exist_ok=True)
            for img_file in member_dir.glob('*.*'):
                shutil.copy(img_file, target_dir)

    print(f"✅ 사진 {COLAB_TEMP_DIR} (Colab 로컬)에 복사 완료.")

except Exception as e:
    print(f"🚨 Drive 연결 또는 복사 실패: {e}")
    COLAB_TEMP_DIR = Path("/content/gdrive/MyDrive/aespa/aespa_photo")

# 파일 경로 및 상수
PHOTO_DIR = COLAB_TEMP_DIR
EMBEDDINGS_FILE = Path("/content/aespa_embeddings_colab.pkl")
TARGET_SIZE = (224, 224)
# 유사도 기준으로 임계값 재설정 (거리가 0.5일 때 유사도 75% -> 75% 미만이면 신뢰도 낮음)
SIMILARITY_THRESHOLD = distance_to_similarity_percent(0.5)

# ----------------------------------------------------
# 2. VGG16 모델 로드 및 임베딩 로직 (GPU 오류 우회 적용)
# ----------------------------------------------------
tf.keras.backend.clear_session()

with tf.device('/CPU:0'):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(TARGET_SIZE[0], TARGET_SIZE[1], 3))

def create_vgg_embedding_model(model):
    x = tf.keras.layers.GlobalAveragePooling2D()(model.output)
    return Model(inputs=model.input, outputs=x)

with tf.device('/CPU:0'):
    VGG_EMBEDDING_MODEL = create_vgg_embedding_model(base_model)

def preprocess_image(img_path_or_pil):
    if isinstance(img_path_or_pil, Path):
        img = Image.open(str(img_path_or_pil)).convert('RGB')
    else: img = img_path_or_pil.convert('RGB')
    img = img.resize(TARGET_SIZE)
    img_array = np.array(img).astype('float32')
    img_array = tf.keras.applications.vgg16.preprocess_input(img_array)
    return np.expand_dims(img_array, axis=0)

def get_embedding(model, preprocessed_image_array):
    with tf.device('/CPU:0'):
        embedding = model.predict(preprocessed_image_array, verbose=0)
    embedding = embedding / np.linalg.norm(embedding, axis=1, keepdims=True)
    return embedding[0]

def create_and_save_embeddings(model, photo_dir, output_file):
    if output_file.exists() and output_file.is_file():
        try:
            with open(output_file, 'rb') as f: return pickle.load(f)
        except:
            os.remove(output_file)
    member_embeddings = {}
    if not photo_dir.is_dir(): return {}
    for member_name_dir in photo_dir.iterdir():
        if member_name_dir.is_dir():
            member_name = member_name_dir.name
            embeddings = []
            photo_paths = []
            for img_path in member_name_dir.glob('*.*'):
                try:
                    preprocessed_img = preprocess_image(img_path)
                    embedding = get_embedding(model, preprocessed_img)
                    embeddings.append(embedding)
                    photo_paths.append(str(img_path))
                except: pass
            if embeddings:
                member_embeddings[member_name] = {"embeddings": np.array(embeddings), "photo_paths": photo_paths}
    if member_embeddings:
        with open(output_file, 'wb') as f: pickle.dump(member_embeddings, f)
        return member_embeddings
    return {}

# --- 메인 데이터 로드 ---
MEMBER_EMBEDDINGS_DATA = create_and_save_embeddings(VGG_EMBEDDING_MODEL, PHOTO_DIR, EMBEDDINGS_FILE)

# ----------------------------------------------------
# 4. Gradio 분석 함수 (유사도 퍼센트 출력)
# ----------------------------------------------------
def analyze_similarity(image_pil):
    if image_pil is None or not MEMBER_EMBEDDINGS_DATA:
        return "⚠️ 오류: 초기화 또는 사진 업로드 실패.", None

    try:
        user_preprocessed = preprocess_image(image_pil)
        user_embedding = get_embedding(VGG_EMBEDDING_MODEL, user_preprocessed)
    except Exception as e:
        return f"❌ 사용자 이미지 처리 실패: 이미지 파일에 문제가 있습니다. ({e})", None

    # 모든 멤버의 최소 유사도 정보를 저장할 딕셔너리
    all_max_similarities = {}
    best_match = {"member": "미정", "distance": 1.0, "photo_path": None}

    for member_name, data in MEMBER_EMBEDDINGS_DATA.items():
        member_embeddings = data["embeddings"]

        distances = [cosine(emb, user_embedding) for emb in member_embeddings]
        min_dist = np.min(distances)
        min_dist_index = np.argmin(distances)

        # 거리를 유사도 %로 변환
        max_similarity_percent = distance_to_similarity_percent(min_dist)
        all_max_similarities[member_name] = max_similarity_percent

        # 가장 거리가 가까운 (유사도가 가장 높은) 멤버 찾기
        if min_dist < best_match["distance"]:
            min_dist_photo_path = data["photo_paths"][min_dist_index]
            best_match.update({
                "member": member_name,
                "distance": min_dist, # 거리는 내부 계산용으로 유지
                "photo_path": min_dist_photo_path
            })

    final_output_markdown = ""
    best_member_name = best_match["member"]
    best_member_korean = KOREAN_NAMES.get(best_member_name, best_member_name)

    matched_photo_path = best_match["photo_path"]
    best_similarity = all_max_similarities[best_member_name]

    confidence_warning = ""
    # 유사도 임계값(75%)보다 낮으면 신뢰도 경고 표시
    if best_similarity < SIMILARITY_THRESHOLD:
        confidence_warning = f"\n⚠️ **주의:** 분석 결과의 신뢰도가 낮을 수 있습니다. (유사도: {best_similarity:.2f}%)"
    else:
        # 최고 유사도 값을 직접 출력 메시지에 포함
        confidence_warning = f"\n✅ **최고 유사도:** {best_similarity:.2f}%"


    # 1. 최상단 간결 메시지 출력 (유사도 퍼센트 포함)
    final_output_markdown += f"## 제일 닮은 사람은 **{best_member_korean}** 입니다."
    final_output_markdown += confidence_warning
    final_output_markdown += "\n가장 유사한 사진을 찾았습니다.\n"

    # 2. 💡 유사도 퍼센트 기반 분석 표 추가
    final_output_markdown += "\n### 📊 멤버별 최대 유사도 분석 결과\n"
    final_output_markdown += "| 멤버 | 최대 유사도 (%) (↑) |\n" # 유사도는 높을수록 닮음 (↑ 표시)
    final_output_markdown += "|---|---|\n"

    # 유사도가 높은 순서대로 정렬
    sorted_similarities = sorted(all_max_similarities.items(), key=lambda item: item[1], reverse=True)

    for member_name, sim_percent in sorted_similarities:
        member_name_korean = KOREAN_NAMES.get(member_name, member_name)
        # 가장 닮은 멤버는 굵게 표시
        if member_name == best_member_name:
             final_output_markdown += f"| **{member_name_korean}** | **{sim_percent:.2f}%** |\n"
        else:
             final_output_markdown += f"| {member_name_korean} | {sim_percent:.2f}% |\n"

    final_output_markdown += "\n* "


    return final_output_markdown, matched_photo_path

# ----------------------------------------------------
# 5. Gradio 인터페이스 실행 (출력 항목 변경 없음)
# ----------------------------------------------------
if MEMBER_EMBEDDINGS_DATA:
    print("\n\n🎉🎉 분석 준비 완료! Gradio 앱 실행 중...")

    original_cwd = os.getcwd()
    os.chdir('/content')
    print("DEBUG: 작업 디렉토리를 /content로 변경했습니다.")


    iface = gr.Interface(
        fn=analyze_similarity,
        inputs=gr.Image(type="pil", label="📸 분석할 사진을 업로드하세요"),
        outputs=[
            gr.Markdown(label="분석 결과"),
            # 사진 크기 확대 (300x300)
            gr.Image(label="가장 닮은 멤버 (가장 유사한 사진)", type="filepath", width=300, height=300)
        ],
        # Gradio 제목 변경
        title="에스파 닮은 꼴 찾기",
        description="가장 닮은 에스파 멤버를 찾아 드립니다.",
        theme=gr.themes.Soft()
    )

    print("\n\n🚀 Gradio 앱 실행 중... 공개 링크(Public URL)를 확인하세요.")
    iface.launch(share=True, debug=False, show_error=True)

    os.chdir(original_cwd)

else:
    print("\n\n🚨🚨 Gradio 앱 실행 실패. 런타임 및 Colab 로그를 확인하세요.")

  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
Mounted at /content/gdrive
✅ 사진 /content/temp_aespa_photos (Colab 로컬)에 복사 완료.


🎉🎉 분석 준비 완료! Gradio 앱 실행 중...
DEBUG: 작업 디렉토리를 /content로 변경했습니다.


🚀 Gradio 앱 실행 중... 공개 링크(Public URL)를 확인하세요.
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
IMPORTANT: You are using gradio version 3.50.2, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://07cfd1f211b229ed8b.grad

In [ ]:
-